In [2]:
import os
import os.path
import numpy as np
import h5py

In [3]:
# Define which folders to parse:
root_dir='/esat/opal/kkelchte/docker_home/pilot_data/canyon_rl'

data_dirs=sorted([d for d in os.listdir(root_dir) if d.startswith('0')])


In [4]:
f = h5py.File('mytestfile.hdf5', 'w')
dset = f.create_dataset("mydataset", (100,), dtype='i')
